In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipyvolume as ipv
%matplotlib inline

from scipy.spatial import Voronoi
from scipy import interpolate

from tyssue import Sheet, Monolayer, ClosedMonolayerGeometry
from tyssue import SheetGeometry
from tyssue.geometry.sheet_geometry import ClosedSheetGeometry
from tyssue.generation import hexa_cylinder, sheet_from_cell_centers, extrude
from tyssue.topology import type1_transition
from tyssue.draw import sheet_view

from tyssue.dynamics import model_factory, effectors
from tyssue.solvers.quasistatic import QSSolver
from tyssue import config

import math


### Initial tissue generation

In [10]:
points_xyz = hexa_cylinder(42, 61, 30, noise=0, capped=True)
sheet = sheet_from_cell_centers(points_xyz)
ClosedSheetGeometry.update_all(sheet)


In [11]:
ipv.clear()
fig, mesh = sheet_view(
    sheet,
    mode='3D',
    face={
        "visible":True,
        "color": sheet.face_df.area
    }
)
ipv.squarelim()
ipv.show()

VBox(children=(Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], meshes=[Mesh(color=array('#2b5d0a', dtype='<U7'), lines=array([[   0,  258],
       [3049, 3046],
       [1971, 3049],
       ...,
       [1583, 1582],
       [ 685, 1157],
       [3737, 3734]], dtype=uint32), texture=None, x=array([-18.16537952,  29.55704035,   6.93202784, ..., -18.12016916,
       -18.11913319, -19.29156804]), y=array([ 1.28207224, -5.18413972, -0.28866288, ..., 23.79183553,
       23.7904887 , 22.86187688]), z=array([-135.80200748,  -54.96686099, -135.7257545 , ...,  -50.47727501,
        -45.98746892,  -48.23241574])), Mesh(color=array([[0.175707, 0.6979  , 0.491033],
       [0.120638, 0.625828, 0.533488],
       [0.166383, 0.690856, 0.496502],
       ...,
       [0.166383, 0.690856, 0.496502],
       [0.166383, 0.690856, 0.496502],
       [0.166383, 0.690856, 0.496502]]), texture=None, triangles=array([[    0,  2816, 19700],
       [ 1495,  2817, 19701],
       [ 1495,  2818, 19702],
       ...,
       [  720, 19697, 36581],
       [  718, 19698, 36582],
       [  719, 19699, 36583]], dtype=uint32), x=array([-21.05269215, -16.69082159, -16.37712151, ...,  11.65626109,
        15.64684987,  11.65749472]), y=array([  0.80211301,  -0.82796118,   3.5951899 , ..., -27.49596293,
       -25.46114984, -27.49891323]), z=array([-135.75409645, -135.78958227, -135.78780585, ...,  -37.00824458,
        -21.30723856,  -14.5801341 ]))], style={'axes': {'visible': False, 'label': {'color': 'white'}, 'ticklabel': {'color': 'white'}, 'color': 'white'}, 'box': {'visible': False}, 'background-color': 'black'}, tf=None, width=400, xlim=[-150.58069334409493, 150.3291998615686], ylim=[-150.58069334409493, 150.3291998615686], zlim=[-150.58069334409493, 150.3291998615686]),))

In [12]:
smodel = model_factory([
    effectors.LumenVolumeElasticity,
    effectors.FaceAreaElasticity,
    effectors.FaceContractility])


dyn_specs = {
    "settings": {
        "lumen_prefered_vol": sheet.settings["lumen_vol"]*1.1,
        "lumen_vol_elasticity": 1e-1/sheet.Nf,
        "threshold_length": 1e-2,
    },
    "face": {
        "area_elasticity": 1.,
        #"prefered_area": sheet.face_df.area.median(), 
        "prefered_area": sheet.face_df.area.median(), 
        "contractility": 1e-4,
    },
    "edge": {
        "ux": 0.0,
        "uy": 0.0,
        "uz": 0.0,
    }
}
## Those settings are set homogenously in the epithelium
sheet.update_specs(dyn_specs, reset=True)
ClosedSheetGeometry.update_all(sheet)

In [13]:
solver = QSSolver(with_t1=False, with_t3=False)
res = solver.find_energy_min(sheet, ClosedSheetGeometry, smodel, options={"gtol": 1e-8})

In [14]:
from tyssue.io import hdf5
hdfname = 'superegg.hf5'
hdf5.save_datasets(hdfname, sheet)

In [15]:
sheet.face_df

,x,y,z,num_sides,area,perimeter,is_alive,face_o,opposite,height,rho,vol,phi,theta,area_elasticity,prefered_area,contractility
face,,,,,,,,,,,,,,,,,
0,-20.778138,1.012832,-132.764598,6,20.075798,16.750973,1,0,-1,16.884867,20.884867,340.677221,3.099779,-0.955847,1.0,20.076221,0.0001
1,-16.927350,-0.846860,-135.107508,6,20.075741,16.736084,1,1,-1,13.060174,17.060174,264.438428,-3.098458,-1.058707,1.0,20.076221,0.0001
2,-16.733447,3.589744,-134.975101,6,20.075797,16.939161,1,2,-1,13.207189,17.207189,265.791936,2.920703,-1.058439,1.0,20.076221,0.0001
3,31.641960,-7.229807,-53.167447,6,20.075858,16.881439,1,3,-1,28.539144,32.539144,572.947215,-0.223806,-0.397823,1.0,20.076221,0.0001
4,32.335033,-2.425710,-51.105151,6,20.075863,16.879092,1,4,-1,28.507541,32.507541,572.313266,-0.074596,-0.383691,1.0,20.076221,0.0001
5,31.582667,-7.215164,-49.040707,6,20.075868,16.876718,1,5,-1,28.477913,32.477913,571.718888,-0.223806,-0.369482,1.0,20.076221,0.0001
6,8.967335,2.393754,-137.636794,6,20.075693,16.939667,1,6,-1,5.541332,9.541332,107.988044,0.177079,-1.282317,1.0,20.076221,0.0001
7,4.518286,1.213884,-138.415374,6,20.075764,16.690147,1,7,-1,1.117117,5.117117,22.162076,0.107326,-1.423292,1.0,20.076221,0.0001
8,7.971232,-2.336380,-137.825282,6,20.075756,16.767602,1,8,-1,4.512378,8.512378,91.514381,-0.365865,-1.305593,1.0,20.076221,0.0001


In [16]:
sheet.settings

{'lumen_prefered_vol': 837810.3141130656,
 'lumen_vol': 837809.9534519882,
 'lumen_vol_elasticity': 3.551136363636364e-05,
 'threshold_length': 0.01}

In [6]:
extruded = extrude(sheet.datasets, method="normals", scale=0.1)

eptm = Monolayer("tube", extruded)
ClosedMonolayerGeometry.update_all(eptm)


In [7]:
ipv.clear()
fig, mesh = sheet_view(eptm, mode='3D', face={"visible":True, "color":eptm.face_df.area})
ipv.squarelim()
ipv.show()

VBox(children=(Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], meshes=[Mesh(color=array('#2b5d0a', dtype='<U7'), lines=array([[ 143,    0],
       [1314, 1313],
       [1312, 1314],
       ...,
       [4367, 9995],
       [9995, 9993],
       [9993, 4365]], dtype=uint32), texture=None, x=array([-0.06549919, -0.45523666, -0.96163355, ..., -0.08423853,
       -0.04665799, -0.08424675]), y=array([-0.06375642, -0.58694241, -0.1665625 , ...,  0.87074595,
        0.87361721,  0.87087584]), z=array([-4.87287952,  4.65801168, -0.48435483, ...,  1.52836231,
        1.59989944,  1.67717833])), Mesh(color=array([[0.974417, 0.90359 , 0.130215],
       [0.974417, 0.90359 , 0.130215],
       [0.974417, 0.90359 , 0.130215],
       ...,
       [0.151918, 0.500685, 0.557587],
       [0.151918, 0.500685, 0.557587],
       [0.151918, 0.500685, 0.557587]]), texture=None, triangles=array([[     0,  22516, 123820],
       [  1135,  22517, 123821],
       [  1135,  22518, 123822],
       ...,
       [ 22515, 123817, 225121],
       [ 22515, 123818, 225122],
       [ 22515, 123819, 225123]], dtype=uint32), x=array([-4.32526886e-02, -1.52367348e-01, -1.57695121e-04, ...,
       -3.36821771e-01, -4.21345121e-01, -4.69251163e-01]), y=array([-1.51097744e-01, -3.84167772e-02,  1.57752447e-04, ...,
       -8.10616686e-01, -7.69567969e-01, -8.57337248e-01]), z=array([-4.86225512, -4.86289463, -4.87294016, ...,  3.38258353,
        3.38446703,  3.38571793]))], style={'axes': {'visible': False, 'label': {'color': 'white'}, 'ticklabel': {'color': 'white'}, 'color': 'white'}, 'box': {'visible': False}, 'background-color': 'black'}, tf=None, width=400, xlim=[-5.3604957019830675, 5.36059972102351], ylim=[-5.3604957019830675, 5.36059972102351], zlim=[-5.3604957019830675, 5.36059972102351]),))

In [8]:
model = model_factory([effectors.LumenVolumeElasticity,
                       effectors.CellVolumeElasticity,
                       effectors.CellAreaElasticity,
                       effectors.FaceContractility])

## Initial model settings
dyn_specs = {
    "settings": {
        "lumen_prefered_vol": eptm.settings["lumen_vol"]*1.2,
        "lumen_vol_elasticity": 10/eptm.Nc,
        "threshold_length": 1e-2,
    },
    "cell": {
        "prefered_vol": eptm.cell_df.vol.mean(),
        "prefered_area": eptm.cell_df.area.mean(),        
        "vol_elasticity": 1.0,
        "area_elasticity": 1.0,
    },
    "face": {
        "contractility": 1e-5,
    },
    "edge": {
        "ux": 0.0,
        "uy": 0.0,
        "uz": 0.0,
    }
}
## Those settings are set homogenously in the epithelium
eptm.update_specs(dyn_specs, reset=True)

print("Model energy (per cell):")
print("------------------------")

energies = model.compute_energy(eptm, full_output=True)
print(*(f"{label.ljust(25)}: \t{np.sum(energy)/eptm.Nc:.3e}"
      for label, energy in zip(model.labels, energies)), sep='\n')


Reseting column contractility of the face dataset with new specs
Reseting column ux of the edge dataset with new specs
Reseting column uy of the edge dataset with new specs
Reseting column uz of the edge dataset with new specs


Model energy (per cell):
------------------------
Lumen volume elasticity  : 	1.210e-05
Volume elasticity        : 	3.226e-09
Area elasticity          : 	2.242e-06
Contractility            : 	7.139e-06


In [9]:
ClosedMonolayerGeometry.update_all(eptm)

In [10]:
solver = QSSolver(with_t1=False, with_t3=False)


# This can be tuned to increase the gradient
# descent precision (at a computation time cost)
optimize_options = {'ftol': 1e-8, 'gtol':1e-8}

res = solver.find_energy_min(
    eptm,
    ClosedMonolayerGeometry,
    model,
    options=optimize_options
)


In [11]:
ipv.clear()

eptm.face_df["visible"] = (eptm.face_df["segment"] == "basal")

fig, mesh = sheet_view(
    eptm,
    mode='3D',
    edge={"visible": True},
    face={
        "visible": True,
        "color": eptm.face_df.area
    }
)

ipv.show()

VBox(children=(Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], meshes=[Mesh(color=array('#2b5d0a', dtype='<U7'), lines=array([[ 143,    0],
       [1314, 1313],
       [1312, 1314],
       ...,
       [4367, 9995],
       [9995, 9993],
       [9993, 4365]], dtype=uint32), texture=None, x=array([-0.06995495, -0.47343638, -1.02788523, ..., -0.0936005 ,
       -0.04898248, -0.09360958]), y=array([-0.06765299, -0.61242638, -0.17887031, ...,  0.94860834,
        0.9520026 ,  0.94873035]), z=array([-4.93728985,  4.69730014, -0.48432203, ...,  1.52687918,
        1.60121117,  1.67569911])), Mesh(color=array([[0.404001, 0.800275, 0.362552],
       [0.404001, 0.800275, 0.362552],
       [0.404001, 0.800275, 0.362552],
       ...,
       [0.377779, 0.791781, 0.377939],
       [0.386433, 0.794644, 0.372886],
       [0.369214, 0.788888, 0.382914]]), texture=None, triangles=array([[    0,  2816, 19700],
       [ 1135,  2817, 19701],
       [ 1135,  2818, 19702],
       ...,
       [ 2505, 19697, 36581],
       [ 2361, 19698, 36582],
       [  779, 19699, 36583]], dtype=uint32), x=array([-4.28586170e-02, -1.52004119e-01, -8.07823291e-05, ...,
        4.77842998e-02, -2.34217115e-01, -3.68870144e-01]), y=array([-1.50608259e-01, -3.84043836e-02,  3.64299480e-05, ...,
        9.53661507e-01,  9.24616976e-01, -8.81824367e-01]), z=array([-4.84156325, -4.84193956, -4.8525797 , ..., -3.46037657,
        2.19632879,  3.38536043]))], style={'axes': {'visible': False, 'label': {'color': 'white'}, 'ticklabel': {'color': 'white'}, 'color': 'white'}, 'box': {'visible': False}, 'background-color': 'black'}, tf=None, width=400, xlim=[-5.43103074481588, 5.431268972880242], ylim=[-5.43103074481588, 5.431268972880242], zlim=[-5.43103074481588, 5.431268972880242]),))

In [12]:
from tyssue.io import hdf5

In [13]:
hdf5.save_datasets('rigged.hf5', eptm)

/home/admin-suz/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->['edge_o', 'srce_o', 'trgt_o', 'face_o', 'cell_o', 'is_valid', 'segment']]

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/admin-suz/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->['face_o', 'opposite', 'height', 'rho', 'vol', 'phi', 'theta', 'area_elasticity', 'prefered_area', 'segment']]

  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
eptm.settings

{'lumen_prefered_vol': 26.28623155275289,
 'lumen_vol': 26.28300027169302,
 'lumen_vol_elasticity': 0.0035511363636363635,
 'threshold_length': 0.01}